In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Input
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [51]:
tf.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>

In [2]:

# Define your RNN model
model = Sequential()
sequence_length = 10
image_height = 1024
image_width = 1024
num_channels=3
input_shape = (sequence_length, image_height, image_width, 3)


In [3]:
# CNN part
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu',  input_shape=(image_height, image_width, num_channels)))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Flatten())


In [62]:
# RNN part
num_features = 2032128
rnn_model = Sequential()
rnn_model.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, num_features)))
# rnn_model.add(LSTM(128)(Flatten()))


In [105]:
num_classes = 4  # Based on Click/ Input text/ Scroll/ Hover
# Combine CNN and RNN
combined_model = Sequential()
combined_model.add(TimeDistributed(cnn_model, input_shape=input_shape))
combined_model.add(rnn_model)
combined_model.add(Dense(num_classes, activation='softmax'))

In [106]:

# Compile the combined model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to see the architecture
combined_model.summary()

# Now, you can train the model using your sequence of images data
# You'll need to prepare your data and labels before fitting the model
# For example, you can use the 'fit' method: combined_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_5 (TimeDi  (None, 10, 2032128)       93248     
 stributed)                                                      
                                                                 
 sequential_10 (Sequential)  (None, 10, 64)            520241408 
                                                                 
 dense_4 (Dense)             (None, 10, 4)             260       
                                                                 
Total params: 520334916 (1.94 GB)
Trainable params: 520334916 (1.94 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
import cv2
import os
from PIL import Image



In [31]:
# Specify the folder containing your images
folder_path = '/Users/khatabook/Learning/kpath/Kpath/screen_recording_utility/screen_shots/20231006231504'

# Initialize a list to store the loaded images
loaded_images = []
target_size = (1024, 1024)
# Loop through the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (you can add more image extensions if needed)
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        # Load the image using OpenCV
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        image = image.resize(target_size)
        newPath = os.path.join(folder_path + '/resize', filename)
        image.save(newPath)


In [8]:
# Specify the folder containing your images
folder_path = '/Users/khatabook/Learning/kpath/Kpath/screen_recording_utility/screen_shots/20231006231504/resize'

# Initialize a list to store the loaded images
loaded_images = []
# Loop through the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (you can add more image extensions if needed)
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        # Load the image using OpenCV
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)

        # Append the loaded image to the list
        if image is not None:
            loaded_images.append(image)

In [9]:
loaded_images[1].shape


(1024, 1024, 3)

In [74]:
images_per_sublist = 10

# Split the list of images into sublists
list_of_lists_of_images = [loaded_images[i:i + images_per_sublist] for i in range(0, len(loaded_images), images_per_sublist)]


In [97]:
len(list_of_lists_of_images)
labels = ['click','click','click', 'input','input','type','type','type','type','type','type','type','click','click']
labels = [0,0,0,1,1,2,2,2,2,2,2,2,3]

In [76]:
x_train = []
y_train = []

In [77]:
for sequence, label in zip(list_of_lists_of_images, labels):
    x_train.append(sequence)  # Append the sequence of images
    y_train.append(label) 

In [79]:
# x_train.pop()

In [80]:
x_train = np.array(x_train)  # Shape: (num_samples, sequence_length, image_height, image_width, num_channels)
x_train = x_train / 255.0 

In [99]:
y_train_1 = to_categorical(labels, num_classes)
y_train_1.shape

(13, 4)

In [108]:

y_train_final = np.reshape(y_train_1, (y_train_1.shape[0], 1, y_train_1.shape[1]))
y_train_final = np.tile(y_train_final, (1, 10, 1))

In [ ]:
combined_model.fit(x_train, y_train_final, epochs=10, batch_size=13)

Epoch 1/10
